# Generating Weather Data 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [65]:
import requests
import pandas as pd

# Define county locations (latitude and longitude)
county_coords = {
    'San Francisco' : {'latitude': 37.7749, 'longitude': -122.4194},
    'San Mateo': {'latitude': 37.5630, 'longitude': -122.3255},
    'Alameda': {'latitude': 37.7652, 'longitude': -122.2416},
    'Contra Costa': {'latitude': 37.9358, 'longitude': -122.0652},
    'Santa Clara': {'latitude': 37.3337, 'longitude': -121.8907}
}

# Function to fetch weather data for a specific county
def fetch_weather_data(county_name, latitude, longitude):
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'start_date': '2020-03-01',
        'end_date': '2020-12-31',
        "hourly": ["temperature_2m", "precipitation", "weather_code", "wind_speed_10m", "wind_direction_10m", "relative_humidity_2m"],
        'timezone': 'America/Los_Angeles'
    }
    
    response = requests.get('https://archive-api.open-meteo.com/v1/archive', params=params)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['hourly'])
        df['county'] = county_name  # Add county name column
        print(f"Data for {county_name} downloaded successfully!")
        return df
    else:
        print(f"Failed to retrieve data for {county_name}. Status code: {response.status_code}")
        return None

# Loop through the counties and fetch their weather data
weather_data_frames = []
for county, coords in county_coords.items():
    df = fetch_weather_data(county, coords['latitude'], coords['longitude'])
    if df is not None:
        weather_data_frames.append(df)

# Combine all weather data into a single DataFrame
weather_data = pd.concat(weather_data_frames, ignore_index=True)

weather_data.head()

Data for San Francisco downloaded successfully!
Data for San Mateo downloaded successfully!
Data for Alameda downloaded successfully!
Data for Contra Costa downloaded successfully!
Data for Santa Clara downloaded successfully!


,time,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,county
0,2020-03-01T00:00,9.0,0.0,1,21.8,301,77,San Francisco
1,2020-03-01T01:00,8.6,0.0,0,19.6,302,77,San Francisco
2,2020-03-01T02:00,8.1,0.0,0,15.3,319,78,San Francisco
3,2020-03-01T03:00,6.9,0.0,1,9.4,358,80,San Francisco
4,2020-03-01T04:00,8.1,0.0,3,7.1,311,78,San Francisco


In [67]:
# Define the weather code mapping
weather_code_mapping = {
    0: 'Clear sky',
    1: 'Mainly clear',
    2: 'Partly cloudy',
    3: 'Overcast',
    45: 'Fog',
    48: 'Depositing rime fog',
    51: 'Light drizzle',
    53: 'Moderate drizzle',
    55: 'Dense drizzle',
    61: 'Slight rain',
    63: 'Moderate rain',
    65: 'Heavy rain',
    66: 'Freezing rain (light)',
    67: 'Freezing rain (heavy)',
    71: 'Slight snow',
    73: 'Moderate snow',
    75: 'Heavy snow',
    95: 'Thunderstorm',
    96: 'Thunderstorm with slight hail',
    99: 'Thunderstorm with heavy hail'
}

# Map the weather codes to their respective descriptions
weather_data['weather_category'] = weather_data['weather_code'].map(weather_code_mapping)

# Split the 'time' column into 'date' and 'hour'
weather_data['date'] = pd.to_datetime(weather_data['time']).dt.date  # Extract the date
weather_data['hour'] = pd.to_datetime(weather_data['time']).dt.hour  # Extract the hour

# Drop the original 'time' column if no longer needed
weather_data = weather_data.drop(columns=['time'])

# Reorder columns (optional)
weather_data = weather_data[['date', 'hour', 'temperature_2m', 'precipitation', 
                             'weather_code', 'wind_speed_10m', 'wind_direction_10m', 
                             'relative_humidity_2m', 'weather_category', 'county']]

# Save the combined data to a CSV file
weather_data.to_csv('weather_data_all_counties_2020.csv', index=False)
weather_data.head()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
0,2020-03-01,0,9.0,0.0,1,21.8,301,77,Mainly clear,San Francisco
1,2020-03-01,1,8.6,0.0,0,19.6,302,77,Clear sky,San Francisco
2,2020-03-01,2,8.1,0.0,0,15.3,319,78,Clear sky,San Francisco
3,2020-03-01,3,6.9,0.0,1,9.4,358,80,Mainly clear,San Francisco
4,2020-03-01,4,8.1,0.0,3,7.1,311,78,Overcast,San Francisco


In [69]:
weather_data.isnull().sum()

date                    0
hour                    0
temperature_2m          0
precipitation           0
weather_code            0
wind_speed_10m          0
wind_direction_10m      0
relative_humidity_2m    0
weather_category        0
county                  0
dtype: int64

In [71]:
weather_data.tail()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
36715,2020-12-31,19,9.1,0.0,1,5.4,8,81,Mainly clear,Santa Clara
36716,2020-12-31,20,9.9,0.0,1,3.8,343,79,Mainly clear,Santa Clara
36717,2020-12-31,21,9.5,0.0,1,3.3,276,81,Mainly clear,Santa Clara
36718,2020-12-31,22,7.9,0.0,1,5.5,293,85,Mainly clear,Santa Clara
36719,2020-12-31,23,9.0,0.0,2,3.8,287,83,Partly cloudy,Santa Clara


# Combining Ridership Data from 2020 to 2024

In [5]:
import pandas as pd
import numpy as np
ridership_columns = ['Date', 'Hour (24-hour clock)', 'Origin Station', 'Destination Station', 'Number of Exits']

In [7]:
rd = pd.read_csv('date-hour-soo-dest-2020.csv', names=ridership_columns, header=None)

In [9]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5811863 entries, 0 to 5811862
Data columns (total 5 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Date                  object
 1   Hour (24-hour clock)  int64 
 2   Origin Station        object
 3   Destination Station   object
 4   Number of Exits       int64 
dtypes: int64(2), object(3)
memory usage: 221.7+ MB


In [11]:
rd['Date'] = pd.to_datetime(rd['Date'])

In [13]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5811863 entries, 0 to 5811862
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Date                  datetime64[ns]
 1   Hour (24-hour clock)  int64         
 2   Origin Station        object        
 3   Destination Station   object        
 4   Number of Exits       int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 221.7+ MB


In [15]:
rd.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
0,2020-01-01,0,12TH,16TH,5
1,2020-01-01,0,12TH,24TH,3
2,2020-01-01,0,12TH,ASHB,6
3,2020-01-01,0,12TH,BAYF,1
4,2020-01-01,0,12TH,CIVC,3


In [17]:
rd.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
5811858,2020-12-31,23,FRMT,BERY,1
5811859,2020-12-31,23,FRMT,MLPT,1
5811860,2020-12-31,23,MCAR,ANTC,1
5811861,2020-12-31,23,MLBR,MLBR,1
5811862,2020-12-31,23,POWL,BERY,1


In [19]:
rd_new = rd[rd['Date'] >= '2020-03-01']

In [21]:
rd_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4277678 entries, 1534185 to 5811862
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Date                  datetime64[ns]
 1   Hour (24-hour clock)  int64         
 2   Origin Station        object        
 3   Destination Station   object        
 4   Number of Exits       int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 195.8+ MB


In [23]:
rd_new.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
5811858,2020-12-31,23,FRMT,BERY,1
5811859,2020-12-31,23,FRMT,MLPT,1
5811860,2020-12-31,23,MCAR,ANTC,1
5811861,2020-12-31,23,MLBR,MLBR,1
5811862,2020-12-31,23,POWL,BERY,1


In [25]:
rd_new.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
1534185,2020-03-01,0,12TH,12TH,3
1534186,2020-03-01,0,12TH,16TH,1
1534187,2020-03-01,0,12TH,24TH,3
1534188,2020-03-01,0,12TH,ASHB,4
1534189,2020-03-01,0,12TH,BALB,1


In [27]:
rd_new.to_csv('date-hour-soo-dest-2020.csv', index=False, header=False)

In [76]:
ridership_columns = ['Date', 'Hour (24-hour clock)', 'Origin Station', 'Destination Station', 'Number of Exits']

In [96]:
rd = pd.read_csv('date-hour-soo-dest-2020.csv', names=ridership_columns, header=None)

In [98]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5811863 entries, 0 to 5811862
Data columns (total 5 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Date                  object
 1   Hour (24-hour clock)  int64 
 2   Origin Station        object
 3   Destination Station   object
 4   Number of Exits       int64 
dtypes: int64(2), object(3)
memory usage: 221.7+ MB


In [100]:
rd['Date'] = pd.to_datetime(rd['Date'])

In [104]:
rd.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
0,2020-01-01,0,12TH,16TH,5
1,2020-01-01,0,12TH,24TH,3
2,2020-01-01,0,12TH,ASHB,6
3,2020-01-01,0,12TH,BAYF,1
4,2020-01-01,0,12TH,CIVC,3


In [112]:
rd.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
5811858,2020-12-31,23,FRMT,BERY,1
5811859,2020-12-31,23,FRMT,MLPT,1
5811860,2020-12-31,23,MCAR,ANTC,1
5811861,2020-12-31,23,MLBR,MLBR,1
5811862,2020-12-31,23,POWL,BERY,1


In [108]:
rd_new = rd[rd['Date'] >= '2020-03-01']

In [110]:
rd_new.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
1534185,2020-03-01,0,12TH,12TH,3
1534186,2020-03-01,0,12TH,16TH,1
1534187,2020-03-01,0,12TH,24TH,3
1534188,2020-03-01,0,12TH,ASHB,4
1534189,2020-03-01,0,12TH,BALB,1


In [114]:
rd_new.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
5811858,2020-12-31,23,FRMT,BERY,1
5811859,2020-12-31,23,FRMT,MLPT,1
5811860,2020-12-31,23,MCAR,ANTC,1
5811861,2020-12-31,23,MLBR,MLBR,1
5811862,2020-12-31,23,POWL,BERY,1


In [116]:
rd_new.to_csv('date-hour-soo-dest-2020.csv', index=False)

In [136]:
rd1 = pd.read_csv('date-hour-soo-dest-2024.csv')

In [138]:
rd1.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
0,2024-01-01,0,12TH,12TH,2
1,2024-01-01,0,12TH,16TH,1
2,2024-01-01,0,12TH,CIVC,1
3,2024-01-01,0,12TH,DELN,1
4,2024-01-01,0,12TH,EMBR,2


In [140]:
rd1.to_csv('date-hour-soo-dest-2024.csv', index=False, header=False)

In [142]:
rd2 = pd.read_csv('date-hour-soo-dest-2024.csv')

In [144]:
rd2.head()

,2024-01-01,0,12TH,12TH.1,2
0,2024-01-01,0,12TH,16TH,1
1,2024-01-01,0,12TH,CIVC,1
2,2024-01-01,0,12TH,DELN,1
3,2024-01-01,0,12TH,EMBR,2
4,2024-01-01,0,12TH,FTVL,1


# Mapping Stations to Counties based on Origin Station

In [206]:
import pandas as pd
import glob

# === Step 1: Load and Concatenate All Ridership Files ===
# Define the column names for ridership data
ridership_columns = ['Date', 'Hour (24-hour clock)', 'Origin Station', 'Destination Station', 'Number of Exits']

# Use glob to load all ridership CSV files
ridership_files = glob.glob('date-hour-soo-dest-*.csv')  # Assuming files are named like 'ridership_2018.csv', 'ridership_2019.csv', etc.
ridership_data = pd.concat([pd.read_csv(file, names=ridership_columns) for file in ridership_files])

# Convert 'Date' to datetime format and extract 'year' column
ridership_data['Date'] = pd.to_datetime(ridership_data['Date'])
ridership_data['Year'] = ridership_data['Date'].dt.year

# Map origin stations to counties (using your station_to_county dictionary)
station_to_county = {
    'EMBR': 'San Francisco', 'MONT': 'San Francisco', 'POWL': 'San Francisco', 'CIVC': 'San Francisco',
    '16TH': 'San Francisco', '24TH': 'San Francisco', 'GLEN': 'San Francisco', 'BALB': 'San Francisco',
    'DALY': 'San Mateo', 'COLM': 'San Mateo', 'SSAN': 'San Mateo', 'SBRN': 'San Mateo',
    'SFIA': 'San Mateo', 'MLBR': 'San Mateo',
    'RICH': 'Alameda', 'DELN': 'Alameda', 'PLZA': 'Alameda', 'NBRK': 'Alameda', 'DBRK': 'Alameda',
    'ASHB': 'Alameda', 'MCAR': 'Alameda', '19TH': 'Alameda', '12TH': 'Alameda', 'LAKE': 'Alameda',
    'FTVL': 'Alameda', 'COLS': 'Alameda', 'SANL': 'Alameda', 'BAYF': 'Alameda', 'HAYW': 'Alameda',
    'SHAY': 'Alameda', 'UCTY': 'Alameda', 'FRMT': 'Alameda', 'WARM': 'Alameda', 'WOAK': 'Alameda',
    'OAKL': 'Alameda', 'CAST': 'Alameda',
    'CONC': 'Contra Costa', 'PHIL': 'Contra Costa', 'WCRK': 'Contra Costa', 'LAFY': 'Contra Costa',
    'ORIN': 'Contra Costa', 'ROCK': 'Contra Costa', 'NCON': 'Contra Costa', 'PITT': 'Contra Costa',
    'PCTR': 'Contra Costa', 'ANTC': 'Contra Costa', 'WDUB': 'Contra Costa', 'DUBL': 'Contra Costa',
    'MLPT': 'Santa Clara', 'BERY': 'Santa Clara'
}
ridership_data['county'] = ridership_data['Origin Station'].map(station_to_county)

In [207]:
ridership_data.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits,Year,county
NaN,2024-01-01,0,12TH,12TH.1,2,2024,Alameda
0.0,2024-01-01,0,12TH,16TH,1,2024,Alameda
1.0,2024-01-01,0,12TH,CIVC,1,2024,Alameda
2.0,2024-01-01,0,12TH,DELN,1,2024,Alameda
3.0,2024-01-01,0,12TH,EMBR,2,2024,Alameda


In [208]:
ridership_data.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits,Year,county
6598058.0,2021-12-31,23,WOAK,BAYF,1,2021,Alameda
6598059.0,2021-12-31,23,WOAK,CIVC,4,2021,Alameda
6598060.0,2021-12-31,23,WOAK,MCAR,1,2021,Alameda
6598061.0,2021-12-31,23,WOAK,MONT,1,2021,Alameda
6598062.0,2021-12-31,23,WOAK,POWL,4,2021,Alameda


In [157]:
ridership_columns = ['Date', 'Hour (24-hour clock)', 'Origin Station', 'Destination Station', 'Number of Exits']

In [29]:
df1 = pd.read_csv('date-hour-soo-dest-2020.csv', names=ridership_columns, header=None)
df1.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
0,2020-03-01,0,12TH,12TH,3
1,2020-03-01,0,12TH,16TH,1
2,2020-03-01,0,12TH,24TH,3
3,2020-03-01,0,12TH,ASHB,4
4,2020-03-01,0,12TH,BALB,1


In [31]:
df1.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
4277673,2020-12-31,23,FRMT,BERY,1
4277674,2020-12-31,23,FRMT,MLPT,1
4277675,2020-12-31,23,MCAR,ANTC,1
4277676,2020-12-31,23,MLBR,MLBR,1
4277677,2020-12-31,23,POWL,BERY,1


In [33]:
df2 = pd.read_csv('date-hour-soo-dest-2021.csv', names=ridership_columns, header=None)
df2.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
NaN,2021-01-01,6,CIVC,POWL,2
0.0,2021-01-01,6,EMBR,EMBR,2
1.0,2021-01-01,6,LAKE,POWL,1
2.0,2021-01-01,7,12TH,DBRK,1
3.0,2021-01-01,7,12TH,POWL,1


In [35]:
df2.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
6598058.0,2021-12-31,23,WOAK,BAYF,1
6598059.0,2021-12-31,23,WOAK,CIVC,4
6598060.0,2021-12-31,23,WOAK,MCAR,1
6598061.0,2021-12-31,23,WOAK,MONT,1
6598062.0,2021-12-31,23,WOAK,POWL,4


In [37]:
df3 = pd.read_csv('date-hour-soo-dest-2022.csv', names=ridership_columns, header=None)
df4 = pd.read_csv('date-hour-soo-dest-2023.csv', names=ridership_columns, header=None)
df5 = pd.read_csv('date-hour-soo-dest-2024.csv', names=ridership_columns, header=None)

In [38]:
# Combine the DataFrames in the specified order
combined_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [39]:
combined_df.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
0,2020-03-01,0,12TH,12TH,3
1,2020-03-01,0,12TH,16TH,1
2,2020-03-01,0,12TH,24TH,3
3,2020-03-01,0,12TH,ASHB,4
4,2020-03-01,0,12TH,BALB,1


In [41]:
combined_df.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits
36015723,2024-11-30,23,WOAK,MLBR,1
36015724,2024-11-30,23,WOAK,MONT,1
36015725,2024-11-30,23,WOAK,POWL,7
36015726,2024-11-30,23,WOAK,SANL,1
36015727,2024-11-30,23,WOAK,SFIA,1


In [45]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36015728 entries, 0 to 36015727
Data columns (total 5 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   Date                  object
 1   Hour (24-hour clock)  int64 
 2   Origin Station        object
 3   Destination Station   object
 4   Number of Exits       int64 
dtypes: int64(2), object(3)
memory usage: 1.3+ GB


In [47]:
# Converting 'Date' to datetime format and extract 'year' column
ridership_data = combined_df
ridership_data['Date'] = pd.to_datetime(ridership_data['Date'])
ridership_data['Year'] = ridership_data['Date'].dt.year

# Mapping origin stations to counties (using your station_to_county dictionary)
station_to_county = {
    'EMBR': 'San Francisco', 'MONT': 'San Francisco', 'POWL': 'San Francisco', 'CIVC': 'San Francisco',
    '16TH': 'San Francisco', '24TH': 'San Francisco', 'GLEN': 'San Francisco', 'BALB': 'San Francisco',
    'DALY': 'San Mateo', 'COLM': 'San Mateo', 'SSAN': 'San Mateo', 'SBRN': 'San Mateo',
    'SFIA': 'San Mateo', 'MLBR': 'San Mateo',
    'RICH': 'Alameda', 'DELN': 'Alameda', 'PLZA': 'Alameda', 'NBRK': 'Alameda', 'DBRK': 'Alameda',
    'ASHB': 'Alameda', 'MCAR': 'Alameda', '19TH': 'Alameda', '12TH': 'Alameda', 'LAKE': 'Alameda',
    'FTVL': 'Alameda', 'COLS': 'Alameda', 'SANL': 'Alameda', 'BAYF': 'Alameda', 'HAYW': 'Alameda',
    'SHAY': 'Alameda', 'UCTY': 'Alameda', 'FRMT': 'Alameda', 'WARM': 'Alameda', 'WOAK': 'Alameda',
    'OAKL': 'Alameda', 'CAST': 'Alameda',
    'CONC': 'Contra Costa', 'PHIL': 'Contra Costa', 'WCRK': 'Contra Costa', 'LAFY': 'Contra Costa',
    'ORIN': 'Contra Costa', 'ROCK': 'Contra Costa', 'NCON': 'Contra Costa', 'PITT': 'Contra Costa',
    'PCTR': 'Contra Costa', 'ANTC': 'Contra Costa', 'WDUB': 'Contra Costa', 'DUBL': 'Contra Costa',
    'MLPT': 'Santa Clara', 'BERY': 'Santa Clara'
}
ridership_data['county'] = ridership_data['Origin Station'].map(station_to_county)

In [48]:
ridership_data.head()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits,Year,county
0,2020-03-01,0,12TH,12TH,3,2020,Alameda
1,2020-03-01,0,12TH,16TH,1,2020,Alameda
2,2020-03-01,0,12TH,24TH,3,2020,Alameda
3,2020-03-01,0,12TH,ASHB,4,2020,Alameda
4,2020-03-01,0,12TH,BALB,1,2020,Alameda


In [51]:
ridership_data.tail()

,Date,Hour (24-hour clock),Origin Station,Destination Station,Number of Exits,Year,county
36015723,2024-11-30,23,WOAK,MLBR,1,2024,Alameda
36015724,2024-11-30,23,WOAK,MONT,1,2024,Alameda
36015725,2024-11-30,23,WOAK,POWL,7,2024,Alameda
36015726,2024-11-30,23,WOAK,SANL,1,2024,Alameda
36015727,2024-11-30,23,WOAK,SFIA,1,2024,Alameda


In [53]:
ridership_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36015728 entries, 0 to 36015727
Data columns (total 7 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Date                  datetime64[ns]
 1   Hour (24-hour clock)  int64         
 2   Origin Station        object        
 3   Destination Station   object        
 4   Number of Exits       int64         
 5   Year                  int32         
 6   county                object        
dtypes: datetime64[ns](1), int32(1), int64(2), object(3)
memory usage: 1.7+ GB


In [55]:
ridership_data.isnull().sum()

Date                    0
Hour (24-hour clock)    0
Origin Station          0
Destination Station     0
Number of Exits         0
Year                    0
county                  0
dtype: int64

In [57]:
ridership_data.to_csv('ridership_data_2020_to_2024.csv', index=False)

# Combining Weather Data for all Counties from 2020 to 2024

In [61]:
w1 = pd.read_csv('weather_data_all_counties_2020.csv')
w1.head()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
0,2020-01-01,0,9.0,0.0,0,6.9,152,93,Clear sky,San Francisco
1,2020-01-01,1,8.1,0.0,0,8.4,155,95,Clear sky,San Francisco
2,2020-01-01,2,10.7,0.0,0,4.1,315,91,Clear sky,San Francisco
3,2020-01-01,3,10.4,0.0,0,3.1,45,94,Clear sky,San Francisco
4,2020-01-01,4,8.9,0.0,2,6.1,130,96,Partly cloudy,San Francisco


In [63]:
w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43920 entries, 0 to 43919
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  43920 non-null  object 
 1   hour                  43920 non-null  int64  
 2   temperature_2m        43920 non-null  float64
 3   precipitation         43920 non-null  float64
 4   weather_code          43920 non-null  int64  
 5   wind_speed_10m        43920 non-null  float64
 6   wind_direction_10m    43920 non-null  int64  
 7   relative_humidity_2m  43920 non-null  int64  
 8   weather_category      43920 non-null  object 
 9   county                43920 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 3.4+ MB


In [65]:
w1['date'] = pd.to_datetime(w1['date'])

In [67]:
w11 = w1[w1['date'] >= '2020-03-01']

In [71]:
w11.head()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
1440,2020-03-01,0,9.6,0.0,1,27.6,294,75,Mainly clear,San Francisco
1441,2020-03-01,1,9.0,0.0,0,21.8,301,77,Clear sky,San Francisco
1442,2020-03-01,2,8.6,0.0,0,19.6,302,77,Clear sky,San Francisco
1443,2020-03-01,3,8.1,0.0,0,15.3,319,78,Clear sky,San Francisco
1444,2020-03-01,4,6.9,0.0,1,9.4,358,80,Mainly clear,San Francisco


In [73]:
w11.tail()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
43915,2020-12-31,19,8.7,0.0,0,6.8,342,82,Clear sky,Santa Clara
43916,2020-12-31,20,9.1,0.0,0,5.4,8,81,Clear sky,Santa Clara
43917,2020-12-31,21,9.9,0.0,0,3.8,343,79,Clear sky,Santa Clara
43918,2020-12-31,22,9.5,0.0,0,3.3,276,81,Clear sky,Santa Clara
43919,2020-12-31,23,7.9,0.0,0,5.5,293,85,Clear sky,Santa Clara


In [75]:
w11.to_csv('weather_data_all_counties_2020.csv')

In [73]:
w1 = pd.read_csv('weather_data_all_counties_2020.csv')

In [75]:
w1.head()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
0,2020-03-01,0,9.0,0.0,1,21.8,301,77,Mainly clear,San Francisco
1,2020-03-01,1,8.6,0.0,0,19.6,302,77,Clear sky,San Francisco
2,2020-03-01,2,8.1,0.0,0,15.3,319,78,Clear sky,San Francisco
3,2020-03-01,3,6.9,0.0,1,9.4,358,80,Mainly clear,San Francisco
4,2020-03-01,4,8.1,0.0,3,7.1,311,78,Overcast,San Francisco


In [77]:
w1.tail()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
36715,2020-12-31,19,9.1,0.0,1,5.4,8,81,Mainly clear,Santa Clara
36716,2020-12-31,20,9.9,0.0,1,3.8,343,79,Mainly clear,Santa Clara
36717,2020-12-31,21,9.5,0.0,1,3.3,276,81,Mainly clear,Santa Clara
36718,2020-12-31,22,7.9,0.0,1,5.5,293,85,Mainly clear,Santa Clara
36719,2020-12-31,23,9.0,0.0,2,3.8,287,83,Partly cloudy,Santa Clara


In [79]:
w2 = pd.read_csv('weather_data_all_counties_2021.csv')
w3 = pd.read_csv('weather_data_all_counties_2022.csv')
w4 = pd.read_csv('weather_data_all_counties_2023.csv')
w5 = pd.read_csv('weather_data_all_counties_2024.csv')

In [81]:
combined_w = pd.concat([w1, w2, w3, w4, w5], ignore_index=True)

In [83]:
combined_w.head()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
0,2020-03-01,0,9.0,0.0,1,21.8,301,77,Mainly clear,San Francisco
1,2020-03-01,1,8.6,0.0,0,19.6,302,77,Clear sky,San Francisco
2,2020-03-01,2,8.1,0.0,0,15.3,319,78,Clear sky,San Francisco
3,2020-03-01,3,6.9,0.0,1,9.4,358,80,Mainly clear,San Francisco
4,2020-03-01,4,8.1,0.0,3,7.1,311,78,Overcast,San Francisco


In [85]:
combined_w.tail()

,date,hour,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category,county
208315,2024-11-30,19,10.5,0.0,3,0.8,225,83,Overcast,Santa Clara
208316,2024-11-30,20,9.8,0.0,3,1.2,231,85,Overcast,Santa Clara
208317,2024-11-30,21,9.7,0.0,2,2.9,277,86,Partly cloudy,Santa Clara
208318,2024-11-30,22,8.7,0.0,3,2.6,258,90,Overcast,Santa Clara
208319,2024-11-30,23,8.6,0.0,3,0.6,72,90,Overcast,Santa Clara


In [87]:
weather_data = combined_w
# Converting 'date' column to datetime format
weather_data['date'] = pd.to_datetime(weather_data['date'])

In [89]:
weather_data.to_csv('weather_data_2020_to_2024.csv', index=False)

# Combining Ridership and Weather Datasets

In [91]:
# Merging BART Data with Weather Data ===
# Renameing columns in bart_data to align with weather_data for merging
ridership_data = ridership_data.rename(columns={'Date': 'date', 'Hour (24-hour clock)': 'hour'})

# Merging the two datasets on 'date', 'hour', and 'county'
merged_data = pd.merge(ridership_data, weather_data, on=['date', 'hour', 'county'], how='left')
merged_data.head()

,date,hour,Origin Station,Destination Station,Number of Exits,Year,county,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category
0,2020-03-01,0,12TH,12TH,3,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
1,2020-03-01,0,12TH,16TH,1,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
2,2020-03-01,0,12TH,24TH,3,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
3,2020-03-01,0,12TH,ASHB,4,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
4,2020-03-01,0,12TH,BALB,1,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear


In [93]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36015728 entries, 0 to 36015727
Data columns (total 14 columns):
 #   Column                Dtype         
---  ------                -----         
 0   date                  datetime64[ns]
 1   hour                  int64         
 2   Origin Station        object        
 3   Destination Station   object        
 4   Number of Exits       int64         
 5   Year                  int32         
 6   county                object        
 7   temperature_2m        float64       
 8   precipitation         float64       
 9   weather_code          int64         
 10  wind_speed_10m        float64       
 11  wind_direction_10m    int64         
 12  relative_humidity_2m  int64         
 13  weather_category      object        
dtypes: datetime64[ns](1), float64(3), int32(1), int64(5), object(4)
memory usage: 3.6+ GB


In [95]:
merged_data.isnull().sum()

date                    0
hour                    0
Origin Station          0
Destination Station     0
Number of Exits         0
Year                    0
county                  0
temperature_2m          0
precipitation           0
weather_code            0
wind_speed_10m          0
wind_direction_10m      0
relative_humidity_2m    0
weather_category        0
dtype: int64

In [97]:
merged_data.to_csv('ridership_and_weather_data_combined_2020_to_2024.csv', index=False)

In [99]:
merged_data.columns

Index(['date', 'hour', 'Origin Station', 'Destination Station',
       'Number of Exits', 'Year', 'county', 'temperature_2m', 'precipitation',
       'weather_code', 'wind_speed_10m', 'wind_direction_10m',
       'relative_humidity_2m', 'weather_category'],
      dtype='object')

In [113]:
merged_data.head()

,date,hour,Origin Station,Destination Station,Number of Exits,Year,county,temperature_2m,precipitation,weather_code,wind_speed_10m,wind_direction_10m,relative_humidity_2m,weather_category
0,2020-03-01,0,12TH,12TH,3,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
1,2020-03-01,0,12TH,16TH,1,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
2,2020-03-01,0,12TH,24TH,3,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
3,2020-03-01,0,12TH,ASHB,4,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear
4,2020-03-01,0,12TH,BALB,1,2020,Alameda,9.0,0.0,1,21.8,301,77,Mainly clear


In [109]:
# Grouping by 'date', 'Origin Station', and 'Destination Station' and aggregate
daily_data = merged_data.groupby(["date", "Origin Station", "Destination Station"], as_index=False).agg({
    "Number of Exits": "sum",           # Total daily ridership for each origin-destination pair
    "temperature_2m": "mean",           # Average daily temperature at the origin station
    "precipitation": "sum",             # Total daily precipitation at the origin station
    "wind_speed_10m": "mean",           # Average wind speed at the origin station
    "wind_direction_10m": "mean",       # Average wind direction at the origin station
    "relative_humidity_2m": "mean"      # Average relative humidity at the origin station
})

# Displaying the first few rows of the resulting dataframe
print(daily_data.head())

        date Origin Station Destination Station  Number of Exits  \
0 2020-03-01           12TH                12TH               35   
1 2020-03-01           12TH                16TH               92   
2 2020-03-01           12TH                19TH               13   
3 2020-03-01           12TH                24TH               95   
4 2020-03-01           12TH                ANTC               13   

   temperature_2m  precipitation  wind_speed_10m  wind_direction_10m  \
0       11.221429            0.0       12.628571          214.000000   
1       11.706667            0.0       13.480000          198.866667   
2       11.462500            0.0        9.550000          245.250000   
3       11.770588            0.0       14.911765          176.235294   
4       12.533333            0.0       13.216667          162.500000   

   relative_humidity_2m  
0             55.642857  
1             52.800000  
2             54.750000  
3             51.470588  
4             47.000000  


In [119]:
d1 = daily_data[daily_data['date'] == '2020-03-01']

In [121]:
d1

,date,Origin Station,Destination Station,Number of Exits,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,relative_humidity_2m
0,2020-03-01,12TH,12TH,35,11.221429,0.0,12.628571,214.000000,55.642857
1,2020-03-01,12TH,16TH,92,11.706667,0.0,13.480000,198.866667,52.800000
2,2020-03-01,12TH,19TH,13,11.462500,0.0,9.550000,245.250000,54.750000
3,2020-03-01,12TH,24TH,95,11.770588,0.0,14.911765,176.235294,51.470588
4,2020-03-01,12TH,ANTC,13,12.533333,0.0,13.216667,162.500000,47.000000
...,...,...,...,...,...,...,...,...,...
2204,2020-03-01,WOAK,UCTY,6,10.916667,0.0,8.133333,272.166667,56.833333
2205,2020-03-01,WOAK,WARM,4,13.566667,0.0,14.233333,219.666667,45.666667
2206,2020-03-01,WOAK,WCRK,4,11.325000,0.0,10.350000,112.750000,55.000000
2207,2020-03-01,WOAK,WDUB,6,12.240000,0.0,9.280000,247.000000,49.600000


In [125]:
# Aggregating weather data by 'date' and 'Origin Station'
weather_data_1 = merged_data.groupby(["date", "Origin Station"], as_index=False).agg({
    "temperature_2m": "mean",           # Average daily temperature for the origin station
    "precipitation": "sum",             # Total daily precipitation for the origin station
    "wind_speed_10m": "mean",           # Average wind speed for the origin station
    "wind_direction_10m": "mean",       # Average wind direction for the origin station
    "relative_humidity_2m": "mean"      # Average relative humidity for the origin station
})

# Aggregating number of exits by 'date', 'Origin Station', and 'Destination Station'
ridership_data_1 = merged_data.groupby(["date", "Origin Station", "Destination Station"], as_index=False).agg({
    "Number of Exits": "sum"  # Total daily ridership for each origin-destination pair
})

# Merging the weather data back to the ridership data
daily_data = ridership_data_1.merge(weather_data_1, on=["date", "Origin Station"], how="left")

# Displaying the resulting dataset
print(daily_data.head())


        date Origin Station Destination Station  Number of Exits  \
0 2020-03-01           12TH                12TH               35   
1 2020-03-01           12TH                16TH               92   
2 2020-03-01           12TH                19TH               13   
3 2020-03-01           12TH                24TH               95   
4 2020-03-01           12TH                ANTC               13   

   temperature_2m  precipitation  wind_speed_10m  wind_direction_10m  \
0       11.820906            0.0       14.169164           190.02439   
1       11.820906            0.0       14.169164           190.02439   
2       11.820906            0.0       14.169164           190.02439   
3       11.820906            0.0       14.169164           190.02439   
4       11.820906            0.0       14.169164           190.02439   

   relative_humidity_2m  
0              51.84669  
1              51.84669  
2              51.84669  
3              51.84669  
4              51.84669  


In [127]:
daily_data.tail()

,date,Origin Station,Destination Station,Number of Exits,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,relative_humidity_2m
4033935,2024-11-30,WOAK,UCTY,7,10.257389,0.0,9.781034,23.135468,74.477833
4033936,2024-11-30,WOAK,WARM,1,10.257389,0.0,9.781034,23.135468,74.477833
4033937,2024-11-30,WOAK,WCRK,5,10.257389,0.0,9.781034,23.135468,74.477833
4033938,2024-11-30,WOAK,WDUB,5,10.257389,0.0,9.781034,23.135468,74.477833
4033939,2024-11-30,WOAK,WOAK,9,10.257389,0.0,9.781034,23.135468,74.477833


In [129]:
# Mapping origin stations to counties (using your station_to_county dictionary)
station_to_county = {
    'EMBR': 'San Francisco', 'MONT': 'San Francisco', 'POWL': 'San Francisco', 'CIVC': 'San Francisco',
    '16TH': 'San Francisco', '24TH': 'San Francisco', 'GLEN': 'San Francisco', 'BALB': 'San Francisco',
    'DALY': 'San Mateo', 'COLM': 'San Mateo', 'SSAN': 'San Mateo', 'SBRN': 'San Mateo',
    'SFIA': 'San Mateo', 'MLBR': 'San Mateo',
    'RICH': 'Alameda', 'DELN': 'Alameda', 'PLZA': 'Alameda', 'NBRK': 'Alameda', 'DBRK': 'Alameda',
    'ASHB': 'Alameda', 'MCAR': 'Alameda', '19TH': 'Alameda', '12TH': 'Alameda', 'LAKE': 'Alameda',
    'FTVL': 'Alameda', 'COLS': 'Alameda', 'SANL': 'Alameda', 'BAYF': 'Alameda', 'HAYW': 'Alameda',
    'SHAY': 'Alameda', 'UCTY': 'Alameda', 'FRMT': 'Alameda', 'WARM': 'Alameda', 'WOAK': 'Alameda',
    'OAKL': 'Alameda', 'CAST': 'Alameda',
    'CONC': 'Contra Costa', 'PHIL': 'Contra Costa', 'WCRK': 'Contra Costa', 'LAFY': 'Contra Costa',
    'ORIN': 'Contra Costa', 'ROCK': 'Contra Costa', 'NCON': 'Contra Costa', 'PITT': 'Contra Costa',
    'PCTR': 'Contra Costa', 'ANTC': 'Contra Costa', 'WDUB': 'Contra Costa', 'DUBL': 'Contra Costa',
    'MLPT': 'Santa Clara', 'BERY': 'Santa Clara'
}
daily_data['county'] = daily_data['Origin Station'].map(station_to_county)

In [131]:
daily_data.head()

,date,Origin Station,Destination Station,Number of Exits,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,relative_humidity_2m,county
0,2020-03-01,12TH,12TH,35,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
1,2020-03-01,12TH,16TH,92,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
2,2020-03-01,12TH,19TH,13,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
3,2020-03-01,12TH,24TH,95,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
4,2020-03-01,12TH,ANTC,13,11.820906,0.0,14.169164,190.02439,51.84669,Alameda


In [133]:
daily_data.tail()

,date,Origin Station,Destination Station,Number of Exits,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,relative_humidity_2m,county
4033935,2024-11-30,WOAK,UCTY,7,10.257389,0.0,9.781034,23.135468,74.477833,Alameda
4033936,2024-11-30,WOAK,WARM,1,10.257389,0.0,9.781034,23.135468,74.477833,Alameda
4033937,2024-11-30,WOAK,WCRK,5,10.257389,0.0,9.781034,23.135468,74.477833,Alameda
4033938,2024-11-30,WOAK,WDUB,5,10.257389,0.0,9.781034,23.135468,74.477833,Alameda
4033939,2024-11-30,WOAK,WOAK,9,10.257389,0.0,9.781034,23.135468,74.477833,Alameda


In [141]:
daily_data['Number of Exits'].max()

2942

In [139]:
daily_data.describe()

,date,Number of Exits,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,relative_humidity_2m
count,4033940,4.033940e+06,4.033940e+06,4.033940e+06,4.033940e+06,4.033940e+06,4.033940e+06
mean,2022-08-05 19:26:55.304936448,4.415297e+01,1.504761e+01,2.772108e+01,1.604633e+01,2.197755e+02,7.235939e+01
min,2020-03-01 00:00:00,1.000000e+00,4.173029e+00,0.000000e+00,9.500000e-01,1.630556e+01,1.057212e+01
25%,2021-06-11 00:00:00,5.000000e+00,1.200056e+01,0.000000e+00,1.078993e+01,2.036925e+02,6.576404e+01
50%,2022-08-12 00:00:00,1.400000e+01,1.478550e+01,0.000000e+00,1.518258e+01,2.341015e+02,7.525462e+01
75%,2023-10-07 00:00:00,4.400000e+01,1.733132e+01,0.000000e+00,2.063801e+01,2.537299e+02,8.151188e+01
max,2024-11-30 00:00:00,2.942000e+03,4.010000e+01,3.704500e+03,4.647820e+01,3.506852e+02,9.963636e+01
std,NaN,8.491414e+01,4.085437e+00,1.216370e+02,6.735541e+00,5.740506e+01,1.307838e+01


In [143]:
daily_data.columns

Index(['date', 'Origin Station', 'Destination Station', 'Number of Exits',
       'temperature_2m', 'precipitation', 'wind_speed_10m',
       'wind_direction_10m', 'relative_humidity_2m', 'county'],
      dtype='object')

In [145]:
daily_data.isnull().sum()

date                    0
Origin Station          0
Destination Station     0
Number of Exits         0
temperature_2m          0
precipitation           0
wind_speed_10m          0
wind_direction_10m      0
relative_humidity_2m    0
county                  0
dtype: int64

In [147]:
daily_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4033940 entries, 0 to 4033939
Data columns (total 10 columns):
 #   Column                Dtype         
---  ------                -----         
 0   date                  datetime64[ns]
 1   Origin Station        object        
 2   Destination Station   object        
 3   Number of Exits       int64         
 4   temperature_2m        float64       
 5   precipitation         float64       
 6   wind_speed_10m        float64       
 7   wind_direction_10m    float64       
 8   relative_humidity_2m  float64       
 9   county                object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 307.8+ MB


In [149]:
daily_data.to_csv('Daily_Data_2020_2024.csv', index=False)

In [151]:
dd = pd.read_csv('Daily_Data_2020_2024.csv')

In [153]:
dd.head()

,date,Origin Station,Destination Station,Number of Exits,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,relative_humidity_2m,county
0,2020-03-01,12TH,12TH,35,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
1,2020-03-01,12TH,16TH,92,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
2,2020-03-01,12TH,19TH,13,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
3,2020-03-01,12TH,24TH,95,11.820906,0.0,14.169164,190.02439,51.84669,Alameda
4,2020-03-01,12TH,ANTC,13,11.820906,0.0,14.169164,190.02439,51.84669,Alameda


In [155]:
dd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4033940 entries, 0 to 4033939
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date                  object 
 1   Origin Station        object 
 2   Destination Station   object 
 3   Number of Exits       int64  
 4   temperature_2m        float64
 5   precipitation         float64
 6   wind_speed_10m        float64
 7   wind_direction_10m    float64
 8   relative_humidity_2m  float64
 9   county                object 
dtypes: float64(5), int64(1), object(4)
memory usage: 307.8+ MB
